In [ ]:
import numpy as np
from numpy import sin, cos
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.optimize import fsolve

In [ ]:
save_time = 5

In [ ]:
left_x = -150
right_x = 150
left_y = -100
right_y = 100

# lattice-lattice parameters
m_1 = 1
m_2 = 0.5
c = 0.75
a = 1

# wave packet parameters
gamma = 0
omega = 1
beta_x = 0.1
beta_y = 0.1
n_0 = -25
v_0 = -35
u_0 = 1

# integration parameters
dt = 0.05
t_max = 135

In [ ]:
num_x = np.round(np.arange(left_x, right_x, a)/a)
num_y = np.round(np.arange(left_y, right_y, a)/a)
mass = np.zeros(shape=(len(num_y),len(num_x)))

for i, num_i in enumerate(num_x):
    for j, num_j in enumerate(num_y):
        if num_i < 0:
            mass[j, i] = m_1
        else:
            mass[j, i] = m_2
mass

In [ ]:
def energy(mass, stiffness, vel, disp):
    e = mass/2 * vel**2 + stiffness/4 * ((np.roll(disp,-1)-disp)**2+(np.roll(disp,1)-disp)**2+\
                                         (np.roll(disp,-1,axis=0)-disp)**2+(np.roll(disp,1,axis=0)-disp)**2)
    return e

In [ ]:
def specify_initial_and_boundary(num_x, num_y, beta_x, beta_y, n_0, v_0, u_0, c, m_1, a, omega, gamma):
    disp = np.zeros(shape=(len(num_y),len(num_x)), dtype=float)
    vel = np.zeros(shape=(len(num_y),len(num_x)), dtype=float)

    k_1 = fsolve(lambda k: m_1*omega**2-4*c*(sin(cos(gamma)*k)**2+sin(sin(gamma)*k)**2), 1)[0]*2/a
    g_1 = np.sqrt(c/m_1) * (a*cos(gamma)*cos(cos(gamma)*k_1*a/2)*sin(cos(gamma)*k_1*a/2) - \
                            a*sin(gamma)*cos(sin(gamma)*k_1*a/2)*sin(sin(gamma)*k_1*a/2)) / \
                                (np.sqrt((cos(sin(gamma)*k_1*a/2))**2+(sin(cos(gamma)*k_1*a/2))**2))
    
    n_x = np.tile(np.array(num_x),len(num_y)).reshape((len(num_y),-1))
    n_y = np.repeat(np.array(num_y),len(num_x)).reshape((-1,len(num_x)))

    disp = u_0*np.exp(-beta_x**2/2*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma)-v_0*sin(gamma))**2)
    disp *= np.exp(-beta_y**2/2*(-n_x*sin(gamma)+n_y*cos(gamma)+n_0*sin(gamma)-v_0*cos(gamma))**2)
    disp *= sin(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y)
    vel = -u_0*np.exp(-beta_x**2/2*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma)-v_0*sin(gamma))**2)
    vel *= np.exp(-beta_y**2/2*(-n_x*sin(gamma)+n_y*cos(gamma)+n_0*sin(gamma)-v_0*cos(gamma))**2)
    vel *= (omega*cos(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y)-\
            beta_x**2*g_1/a*(n_x*cos(gamma)+n_y*sin(gamma)-n_0*cos(gamma))*\
                sin(k_1*a*cos(gamma)*n_x+k_1*a*sin(gamma)*n_y))

    return disp, vel

In [ ]:
def solver(mass, disp, vel, c, disp_history, vel_history, energy_history, t_max, dt, save_time):

    times = np.arange(0, t_max, dt)

    for t in tqdm(times):

        if t % save_time == 0:
            disp_history += [disp.copy()]
            vel_history += [vel.copy()]
            energy_history += [energy(mass, c, vel, disp)]
        
        # leapfrog synchronized form
        acc1 = (c/mass)*(np.roll(disp,-1)+np.roll(disp,1)+np.roll(disp,-1,axis=0)+np.roll(disp,1,axis=0)-4*disp)
        disp += vel*dt+1/2*acc1*dt**2
        acc2 = (c/mass)*(np.roll(disp,-1)+np.roll(disp,1)+np.roll(disp,-1,axis=0)+np.roll(disp,1,axis=0)-4*disp)
        vel += 1/2*(acc1+acc2)*dt

        # leapfrog self-made form
        #deform = np.roll(disp,-1) + np.roll(disp,1) + np.roll(disp,-1,axis=0) + np.roll(disp,1,axis=0) - 4*disp
        #vel += (c/mass)*deform*dt
        #disp += vel * dt

    return disp_history, vel_history, energy_history

In [ ]:
disp, vel = specify_initial_and_boundary(num_x, num_y, beta_x, beta_y, n_0, v_0, u_0, c, m_1, a, omega, gamma)

fig = plt.figure()
ax = plt.axes(projection="3d")
x, y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(x, y, disp)
plt.show()

In [ ]:
disp_history = []
vel_history = []
energy_history = []
disp_history, vel_history, energy_history = solver(mass, disp, vel, c, disp_history, vel_history,
                                                   energy_history, t_max, dt, save_time)

fig = plt.figure()
ax = plt.axes(projection="3d")
x, y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(x, y, disp_history[-1])
plt.show()

In [ ]:
X, Y = np.meshgrid(num_x, num_y)
levels = np.linspace(disp.min(), disp.max(), 100)
fig, ax = plt.subplots()
cs = ax.contourf(X, Y, disp,levels=levels)
cbar = fig.colorbar(cs)
ax = plt.gca()
ax.set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
#%matplotlib qt
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
X, Y = np.meshgrid(num_x, num_y)
surf = ax.plot_surface(X, Y, disp, cmap=cm.cool)
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()

In [ ]:
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

fig1, ax = plt.subplots()
X, Y = np.meshgrid(num_x, num_y)
levels = np.linspace(energy_history[0].min(), energy_history[0].max(), 100)

cf = ax.contourf(X, Y, energy_history[0], levels=levels)
cbar = fig1.colorbar(cf)
plt.grid()
plt.plot([0,0],[num_y[0],num_y[-1]], c='r', linewidth=0.8)
plt.xlabel('Номер частицы по оси Ox')
plt.ylabel('Номер частицы по оси Oy')
ax.set_aspect('equal', adjustable='box')

def update(frame):
    cf = ax.contourf(X, Y, energy_history[frame], levels=levels)
    plt.title(f't={5*frame}')
    return cf, 

animation.FuncAnimation(fig1, update, frames=len(energy_history))

TODO: изменение colorbar в анимации: [https://stackoverflow.com/questions/39472017/how-to-animate-the-colorbar-in-matplotlib](https://stackoverflow.com/questions/39472017/how-to-animate-the-colorbar-in-matplotlib).

# Comparison of numerical results with analytical solution

In [ ]:
import sympy as sp
from sympy import Symbol, Abs, I, exp, diff
from sympy.plotting import plot


def transmission_analytical(m_1, m_2, c, omega, a, gamma):

    # find wave vector components (in lattice 1) that corresponds to given frequency omega and angle gamma
    k1 = fsolve(lambda k: m_1*omega**2-4*c*(sin(cos(gamma)*k*a/2)**2+sin(sin(gamma)*k*a/2)**2), 1)[0]
    k1_x = k1*cos(gamma)
    k1_y = k1*sin(gamma)

    # find wave vector components (in lattice 2) and refraction angle using Snell's law
    k2_y = k1_y
    k2_x = fsolve(lambda k: m_2*omega**2-4*c*(sin(k*a/2)**2+sin(k2_y*a/2)**2), 0.5)[0]
    k2 = np.sqrt(k2_x**2+k2_y**2)
    zeta = np.arctan(k2_y/k2_x)

    # find group velocities
    k = Symbol('k')
    g1 = diff(2*np.sqrt(c/m_1)*sp.sqrt(sp.sin(k*np.cos(gamma)*a/2)**2+\
                                       sp.sin(k*np.sin(gamma)*a/2)**2),k).evalf(subs={k:k1})
    g1_x = g1*np.cos(gamma)
    g1_y = g1*np.sin(gamma)
    g2 = diff(2*np.sqrt(c/m_2)*sp.sqrt(sp.sin(k*np.cos(zeta)*a/2)**2+\
                                       sp.sin(k*np.sin(zeta)*a/2)**2),k).evalf(subs={k:k2})
    g2_x = g2*np.cos(zeta)
    g2_y = g2*np.sin(zeta)

    A_frac = (exp(I*k1_x*a)-exp(-I*k1_x*a))/(exp(I*k2_x*a)-exp(-I*k1_x*a))
    A_frac = A_frac.evalf()
    trans_coeff = m_2*g2_x/(m_1*g1_x)*(Abs(A_frac))**2
    
    return trans_coeff


def transmission_numerical(m_1, m_2, c, omega, a, gamma):

    #left_x = -400
    #right_x = 600
    #beta_x = 0.03
    #beta_y = 0.03
    #n_0 = -150
    #v_0 = -150
    #u_0 = 1

    num_x = np.round(np.arange(left_x, right_x, a)/a)
    num_y = np.round(np.arange(left_y, right_y, a)/a)
    mass = np.zeros(shape=(len(num_y),len(num_x)))
    for i, num_i in enumerate(num_x):
        for j, num_j in enumerate(num_y):
            if num_i < 0:
                mass[j, i] = m_1
            else:
                mass[j, i] = m_2

    disp, vel = specify_initial_and_boundary(num_x, num_y, beta_x, beta_y, n_0, v_0, u_0, c, m_1, a, omega, gamma)
    
    #dt = 0.01
    #t_max = 350
    
    disp_history = []
    vel_history = []
    energy_history = []
    disp_history, vel_history, energy_history = solver(mass, disp, vel, c, disp_history, vel_history,
                                                       energy_history, t_max, dt, save_time)
    
    energy_sum = np.sum(energy_history[-1])
    energy_right = np.sum(energy_history[-1][:,np.where(num_x == 0)[0][0]:])

    trans_coeff = energy_right/energy_sum

    return trans_coeff

In [ ]:
trans_a = lambda x: transmission_analytical(m_1, m_2, c, omega, a, x)
trans_num = lambda x: transmission_numerical(m_1, m_2, c, omega, a, x)


# gamma_values = abs(10/np.logspace(0,1,20)-11)/10 * (np.pi/4)

gamma_values = np.pi * np.array([0, 1/10, 1/8, 1/6, 1/5, 1/4.5, 1/4.2, 1/4.1, 1/4.05, 1/4.005, 1/4.0005,
                                 1/4, 1/3.9, 1/3.8, 1/3.7, 1/3.6, 1/3.5, 1/3.25, 1/3])

trans_a_values = [trans_a(i) for i in gamma_values]
trans_num_values = [trans_num(j) for j in gamma_values]

In [ ]:
plt.close(fig1)
plt.rcParams["animation.html"] = "none"
plt.figure()
plt.plot(gamma_values/np.pi*180, trans_num_values, c='blue', label='Численно')
plt.plot(gamma_values/np.pi*180, trans_a_values, '--', c='red', label='Аналитически')
plt.title('Incidence angle dependence of the transmission coefficient\n'+\
          f'(m1={m_1}, m2={m_2}, c={c}, a={a}, omega={omega})')
plt.xlabel('gamma, degrees')
plt.ylabel('T')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# transmission coefficient with given incidence angle
trans_a(np.pi/4.2)